## Reading in the data

In [1]:
import pandas as pd
import re

In [2]:
pre_data = pd.read_csv("1930_1970_data.csv")

In [3]:
pre_data.head()

,Year,Address,MapLotBlock,Direction,LotArea,PropertyDescription,SaleDate,AV,Foreclosure,dim1,dim2,no_stories,Price_org,price,AV1,AV2
0,1930,"Chrystie st, 201",2:426-27,ws abt 25 s of Stanton,25x100,5-story bk tnt & strs,-10872,15000-23000,NaN,25.0,100.0,5.0,22000,22000.0,15000.0,23000.0
1,1930,"Eldridge st, 85",1:306-26,ws 125 s Grand,25x100,5-story bk tnt & strs,-10868,35000-46000,Yes,25.0,100.0,5.0,36000,36000.0,35000.0,46000.0
2,1930,"Front st, 76",1:32-19,swc Old slip (No. 27),19.3x33.3x19.1x33.6,4-story bk loft & str bldg,-10871,25000-28000,NaN,19.3,33.3,4.0,56500,56500.0,25000.0,28000.0
3,1930,"Henry st, 200-202",1:270-49,ss 47.6 w Clinton,47.6x100,6-story bk tnt & strs,-10870,31000-70000,Yes,47.6,100.0,6.0,42000,42000.0,31000.0,70000.0
4,1930,"Morton st, 5",2:587-28,ns 50 w Bleecker,25x86x25x87,5-story stn tnt & strs,-10958,12500-22500,NaN,25.0,86.0,5.0,32000,32000.0,12500.0,22500.0


In [4]:
pre_data.columns

Index(['Year', 'Address', 'MapLotBlock', 'Direction', 'LotArea',
       'PropertyDescription', 'SaleDate', 'AV', 'Foreclosure', 'dim1', 'dim2',
       'no_stories', 'Price_org', 'price', 'AV1', 'AV2'],
      dtype='object')

In [5]:
pre_data["Address"].head()

0     Chrystie st, 201
1      Eldridge st, 85
2         Front st, 76
3    Henry st, 200-202
4         Morton st, 5
Name: Address, dtype: object

In [6]:
pre_data.dtypes

Year                    object
Address                 object
MapLotBlock             object
Direction               object
LotArea                 object
PropertyDescription     object
SaleDate                object
AV                      object
Foreclosure             object
dim1                   float64
dim2                   float64
no_stories             float64
Price_org               object
price                  float64
AV1                    float64
AV2                    float64
dtype: object

#### Processing Address column

In [7]:
#split the address and append according to the required format
def processAddress(address):
    # incoming address is actually an object type
    address = str(address)
    if "," in address:
        split_ad = address.split(",")
        number = split_ad[1]
        street = split_ad[0]
        # strip the leading whitespaces
        number = number.lstrip()

        # regular expression for handling cases like "number = 510-512"
        match = re.search("\d+-\d+", number)
        if match:
            numbers = match.group().split("-")
            number = re.sub("\d+-\d+", numbers[0], number)

            # mod_number = ""
        # parse the number and expand according to the required format
        """for c in number:
            if c == "E":
                mod_number += "East"
            elif c == "W":
                mod_number += "West"
            else:
                mod_number+=c"""

        pattern = "\s+W|\s+w"
        match = re.search(pattern, number)
        if match:
            number = re.sub(pattern, " West", number)

        pattern = "\sE|\se"
        match = re.search(pattern, number)
        if match:
            number = re.sub(pattern, " East", number)

            # parse the street and perform modifications
        pattern = "\sst|\sST|\ssT|\sSt"
        match = re.search(pattern, street)
        if match:
            street = re.sub(pattern, " street", street)

        pattern = "\sav|\sAV|\saV|\sAv"
        match = re.search(pattern, street)
        if match:
            street = re.sub(pattern, " avenue", street)

        pattern = "\sblvd|\sBLVD"
        match = re.search(pattern, street)
        if match:
            street = re.sub(pattern, " boulevard", street)

        pattern = "\spl|\sPl|\spL|\sPL"
        match = re.search(pattern, street)
        if match:
            street = re.sub(pattern, " place", street)

        pattern = "\sdr|\sDR|\dR|\sDr"
        match = re.search(pattern, street)
        if match:
            street = re.sub(pattern, " drive", street)

        pattern = "\ster|\sTER"
        match = re.search(pattern, street)
        if match:
            street = re.sub(pattern, " terrace", street)

        pattern = "\ssq|\sSQ|\sSq|\ssQ"
        match = re.search(pattern, street)
        if match:
            street = re.sub(pattern, " square", street)

        pattern = "3D|3\sD|3d|3\sd"
        match = re.search(pattern, street)
        if match:
            street = re.sub(pattern, "3rd", street)

        pattern = "2D|2\sD|2d|2\sd"
        match = re.search(pattern, street)
        if match:
            street = re.sub(pattern, "2nd", street)
        
        pattern = "\s[Ww]$"
        match = re.search(pattern, street)
        if match:
            street = re.sub(pattern, " West", street)
        
        pattern = "\s[Ss]$"
        match = re.search(pattern, street)
        if match:
            street = re.sub(pattern, " South", street)

        return number + " " + street

    else:
        return address

In [8]:
pre_data['mod_address'] = pre_data['Address'].apply(processAddress)

In [9]:
temp_data = pre_data[['Address','mod_address']]

In [10]:
temp_data.head()

,Address,mod_address
0,"Chrystie st, 201",201 Chrystie street
1,"Eldridge st, 85",85 Eldridge street
2,"Front st, 76",76 Front street
3,"Henry st, 200-202",200 Henry street
4,"Morton st, 5",5 Morton street


In [11]:
temp_data.to_csv(r'export_dataframe_modAddress2.csv', index = False)

#### Experimentation 

In [12]:
mlb = "5:1289-pt 1t 36"
match = re.search("\d+-[pP]",mlb)
if match:
    blockLot = match.group().split("-")
    block = blockLot[0]
    lot = re.search("\d+$",mlb).group()
        
    mlb = "1"+block+lot
    

print(mlb)

1128936


### Proessing Maplotblock

In [15]:
def processMapLotBlock(mlb):
    mlb = str(mlb)
    match = re.search("\d+-\d+",mlb)
    block = ""
    lot = ""
    if match:
        blockLot = match.group().split("-")
        block = blockLot[0]
        lot = blockLot[1]
    
    elif re.search("\d+-[pP]",mlb):
        match = re.search("\d+-[pP]",mlb)
        blockLot = match.group().split("-")
        block = blockLot[0]
        lot = re.search("\d+$",mlb).group()
    
    elif re.search("\d+:\d+$",mlb):
        match = re.search("\d+:\d+$",mlb)
        blockLot = match.group().split(":")
        block = blockLot[1]
        lot = "1"
        
    else:
        return mlb
        
        #add zeroes to block if required
    zToBlock = 5 - len(block)
    if zToBlock != 0:
        for i in range(zToBlock):
            block = "0" + block
        
    zToLot = 4 - len(lot)
    if zToLot != 0:
        for i in range(zToLot):
            lot = "0" + lot
        
    mlb = "1"+block+lot

    return mlb

#print(processMapLotBlock("1:75-17-18"))

In [16]:
pre_data['mod_maplotblock'] = pre_data['MapLotBlock'].apply(processMapLotBlock)

In [17]:
temp_data = pre_data[['MapLotBlock','mod_maplotblock']]

In [18]:
temp_data.head()

,MapLotBlock,mod_maplotblock
0,2:426-27,1004260027
1,1:306-26,1003060026
2,1:32-19,1000320019
3,1:270-49,1002700049
4,2:587-28,1005870028


In [19]:
temp_data.to_csv(r'export_mbl_df.csv',index=False)

#### Process LotArea

In [7]:
def areaCal(exp):
    exp = str(exp)
    area = 0
    #condition for containing more than 1 lot
    match = re.search("(and)",exp)
    if match:
        lotList = exp.split("and")
        for lot in lotList:
            area += areaCal(lot.strip())
        return area
    
    match = re.search("^(runs)",exp)
    #condition for handling runs
    if match:
        match = re.search("\d+\.*\d+",exp)
        
        
    #condition for running just numbers
    else:
        
        exp = re.sub("X","x",exp)
        dimList = exp.split("x")
        
        if len(dimList) ==2: 
            try:
                width = float(dimList[0])
            except:
                #print("convertion error for width")
                return area
        
            try:
                breadth = float(dimList[1])
            except:
                #print("Convertion error for breadth")
                return area
        
            area = width*breadth
        
        elif len(dimList) ==4:
            for i in range(len(dimList)):
                try:
                    dimList[i] = float(dimList[i])
                except ValueError:
                    #print('one of the dimension is not a number')
                    return area    
            dimList.sort()
            area = dimList[0]*dimList[3]
            
    
    return area        

In [8]:
pre_data['Calculated_area'] = pre_data['LotArea'].apply(areaCal)

In [9]:
pre_data['Calculated_area']

0       2500.000
1       2500.000
2        641.760
3       4760.000
4       2175.000
5          0.000
6          0.000
7          0.000
8       1520.000
9          0.000
10      4890.000
11      4377.440
12         0.000
13      4373.080
14         0.000
15         0.000
16         0.000
17      1575.000
18      2582.500
19      2300.000
20         0.000
21      2472.500
22      1978.000
23      2110.500
24         0.000
25      8482.600
26         0.000
27      3343.674
28      2002.200
29      1694.781
          ...   
4549    7537.500
4550    1297.500
4551    4041.600
4552    3750.600
4553       0.000
4554    1899.450
4555       0.000
4556    1023.040
4557    2203.780
4558    3015.000
4559    2502.750
4560       0.000
4561       0.000
4562    2216.330
4563       0.000
4564       0.000
4565       0.000
4566       0.000
4567    3966.800
4568    2500.000
4569    1199.231
4570       0.000
4571       0.000
4572       0.000
4573       0.000
4574    2142.354
4575    1775.840
4576       0.0

In [10]:
temp_data = pre_data[['LotArea','Calculated_area']]

In [12]:
temp_data.to_csv(r'export_lotArea_df.csv',index=False)